I will use the cnn / dailymail dataset  
This has news articles with the corresponding highlights that act as summaries  
There are over 300,000 articles, each entry has an article and corresponding highlights

In [1]:
!pip install datasets transformers evaluate torch

In [2]:
from datasets import load_dataset, DatasetDict
from transformers import BartTokenizer, BartForConditionalGeneration, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate
import numpy as np

In [3]:
dataset = load_dataset("cnn_dailymail", "3.0.0")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})


In [4]:
subset_train = dataset["train"].shuffle(seed=42).select(range(2000))
subset_validation = dataset["validation"].shuffle(seed=42).select(range(200))

data = DatasetDict({"train": subset_train, "validation": subset_validation})

In [5]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

max_input_length = 1024
max_target_length = 128

def preprocess_function(examples):
    inputs = examples["article"]
    targets = examples["highlights"]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [6]:
tokenized_data = data.map(preprocess_function, batched=True)

In [7]:
!pip install rouge_score
!pip install --upgrade transformers
!pip install --upgrade "accelerate>=0.26.0"

In [9]:
from transformers import Seq2SeqTrainingArguments
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    output_dir="./bart-finetuned-news",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=False,
    bf16=True
)

rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result_rouge = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    result_bleu = bleu.compute(predictions=[pred.split() for pred in decoded_preds],
                               references=[[ref.split()] for ref in decoded_labels])
    result = {**result_rouge, **result_bleu}
    return result

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/var/folders/xx/pq5vf3ts16x6jt7g3y6hrlcc0000gn/T/ipykernel_24887/2343113025.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
results = trainer.evaluate()
print(results)